# Slice/Sliver Analysis based on various filters

This notebook provides an interactive tool to analyze **resource usage** (e.g., SmartNICs, GPUs, FPGAs, Storage) and **slice activity** across the FABRIC testbed.

## Purpose

- Identify the **Top 10 users** based on the number of component-based slivers used within a selected time range.
- Analyze and filter **slices** based on metadata such as state, site, component type, and time window.
- Allow dynamic filtering by:
  - Component type (e.g., SmartNIC, GPU)
  - Component model (e.g., ConnectX-6)
  - Sliver type (e.g., VM, FABNetv4)
  - Slice state (e.g., StableOK, Closing, Dead)
  - Site (include/exclude)
- Enable exporting and visualizing results.

## How it works

### Sliver Analysis
- Slivers are queried from the FABRIC API based on user-specified filters.
- Filters can include component metadata and sliver characteristics.
- Only slivers associated with hardware components are considered.
- The results are grouped by user to identify the **top consumers** of testbed resources.

### Slice Analysis
- Slices are retrieved using interactive time and state filters.
- Filtering supports inclusion/exclusion of specific slice states (e.g., exclude `Dead`, `Closing`).
- Additional filters for component type, component model, and sites may be applied.
- The output enables insight into **slice allocation patterns** and **user engagement** over time.

Use the interactive widgets provided in each section below to customize and execute your queries.


This cell contains part of the data processing or sliver query logic used to filter and rank users by component usage.

In [ ]:
from fabric_reports_client.reports_api import ReportsApi
from datetime import datetime, timezone
import json
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

api =  ReportsApi(base_url="https://reports.fabric-testbed.net/reports", token_file="/Users/kthare10/work/id_token_prod.json")

query_start=None

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

fabric_projects = ['2dd1ffb8-1aff-45cc-a70d-eb93b65cc26b', '4604cab7-41ff-4c1a-a935-0ca6f20cceeb', '6b76128d-c73f-431f-a245-0397586a7d40', '32e7160e-0318-43f5-a4e3-80209f880833', 
                   '75835e68-f91f-474d-8d54-27a576cc252f', '990d8a8b-7e50-4d13-a3be-0f133ffa8653', '04b14c17-e66a-4405-98fc-d737717e2160', '1630021f-0a0c-4792-a241-997f410d36e1', 
                   '7a5adb91-c4c0-4a1c-8021-7b6c56af196f', '06e8d02a-b27f-4437-829e-8378d20e5a08', '7f33ecf0-5dd7-4fd5-b1b7-061367f8bca6']

## Query all Slices

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from pytz import UTC

# === Global DataFrame for reuse ===
df_slices = pd.DataFrame()

# === Default start: June 1, 2023 | Default end: Start of current month ===
default_start = datetime(2023, 6, 1, tzinfo=UTC)
default_end = datetime.now(UTC).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

# === Widgets ===
slice_start_picker = widgets.DatetimePicker(description='Slice Start:', value=default_start)
slice_end_picker = widgets.DatetimePicker(description='Slice End:', value=default_end)
slice_run_button = widgets.Button(description="Query Slices")
slice_status_output = widgets.Output()

# === Display UI ===
display(slice_start_picker, slice_end_picker, slice_run_button, slice_status_output)

# === Callback ===
def run_slice_query(b):
    global df_slices
    slice_status_output.clear_output()

    start_time = slice_start_picker.value
    end_time = slice_end_picker.value

    if not start_time or not end_time or start_time >= end_time:
        with slice_status_output:
            print("⚠️ Please ensure start time is before end time.")
        return

    slice_run_button.disabled = True
    with slice_status_output:
        print(f"⏳ Querying slices from {start_time} to {end_time}...")

    try:
        response = api.query_slices(
            start_time=start_time,
            end_time=end_time,
            fetch_all=True,
            per_page=1000,
            exclude_project_id=fabric_projects
        )

        slices = response.get("data", [])
        df_slices = pd.DataFrame(slices)

        slice_status_output.clear_output()
        with slice_status_output:
            print(f"✅ Query complete. Total slices: {response.get('total', len(df_slices))}")

        if not df_slices.empty:
            display(df_slices)
        else:
            with slice_status_output:
                print("No slices returned.")

    except Exception as e:
        with slice_status_output:
            print(f"❌ Error during query: {e}")

    slice_run_button.disabled = False

# === Bind button click ===
slice_run_button.on_click(run_slice_query)


## Slices by User Email
Uses `df_slices` dataframe produced by the avove query.

In [ ]:
# Step 3: Interactive selection and filtering
def create_user_selector(df):
    user_emails = sorted(df['user_email'].dropna().unique())
    dropdown = widgets.Dropdown(options=user_emails, description="User Email:")
    
    def on_select_user(change):
        selected_user = change['new']
        user_slices = df[df['user_email'] == selected_user]
        display(user_slices[['user_email', 'user_id', 'slice_name', 'slice_id', 'lease_start', 'lease_end']].sort_values(by='lease_start'))

    dropdown.observe(on_select_user, names='value')
    display(dropdown)

# Run everything
create_user_selector(df_slices)

## Slices per Month

`df_slices` produced by previous cell is used to plot these slices.

In [ ]:

# Step 3: Create DataFrame and process dates
df = df_slices

df['lease_start'] = pd.to_datetime(df['lease_start'], format="mixed", utc=True)
df['month'] = df['lease_start'].dt.to_period('M')  # '2025-04'

monthly_counts = df.groupby('month').size().sort_index()
#for month, count in monthly_counts.items():
#    print(f"{month}: {count}")

# Step 6: Plot
monthly_counts.plot(kind='bar', figsize=(12, 6))
plt.title('Number of Slices Created Per Month')
plt.xlabel('Month')
plt.ylabel('Number of Slices')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


## Active Slices per Month

`df_slices` dataframe from the query above is used here too.

In [ ]:
df = df_slices
# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], format="mixed", utc=True)
df['lease_end'] = pd.to_datetime(df['lease_end'], format="mixed", utc=True)

# Generate monthly range
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Count active slices per month
active_counts = []
for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)
    active = df[(df['lease_start'] <= month_end) & (df['lease_end'] >= month_start)]
    active_counts.append(len(active))

# Plot
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), active_counts)
plt.xticks(rotation=45)
plt.title("Active Slices Per Month")
plt.xlabel("Month")
plt.ylabel("Active Slice Count")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Query Slices filtered by Component Types/Component Model

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from pytz import UTC

# === Global DataFrame ===
df_slices_by_component = pd.DataFrame()
components_selected = []

# === Default time window ===
default_start = datetime(2023, 6, 1, tzinfo=UTC)
default_end = datetime.now(UTC).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

# === Widgets ===
slice_start_picker = widgets.DatetimePicker(description='Slice Start:', value=default_start)
slice_end_picker = widgets.DatetimePicker(description='Slice End:', value=default_end)

component_type_select = widgets.SelectMultiple(
    options=["GPU", "SmartNIC", "SharedNIC", "FPGA", "NVME", "Storage"],
    description="Component Type:"
)

component_model_text = widgets.Text(
    description="Component Model(s):",
    placeholder="comma-separated, e.g. ConnectX-6, A100"
)

site_include_text = widgets.Text(description="Include Sites:", placeholder="comma-separated")
site_exclude_text = widgets.Text(description="Exclude Sites:", placeholder="comma-separated")

slice_run_button = widgets.Button(description="Query Slices")
slice_status_output = widgets.Output()

# === Display all widgets ===
display(
    slice_start_picker, slice_end_picker,
    component_type_select, component_model_text,
    site_include_text, site_exclude_text,
    slice_run_button, slice_status_output
)

# === Callback ===
def run_slice_query(b):
    global df_slices_by_component
    global components_selected
    slice_status_output.clear_output()

    start_time = slice_start_picker.value
    end_time = slice_end_picker.value

    if not start_time or not end_time or start_time >= end_time:
        with slice_status_output:
            print("⚠️ Please ensure start time is before end time.")
        return

    # Parse selected filters
    component_types = list(component_type_select.value)
    components_selected = component_types
    component_models = [m.strip() for m in component_model_text.value.split(",") if m.strip()]
    include_sites = [s.strip() for s in site_include_text.value.split(",") if s.strip()]
    exclude_sites = [s.strip() for s in site_exclude_text.value.split(",") if s.strip()]

    # Build query params
    query_params = {
        "start_time": start_time,
        "end_time": end_time,
        "fetch_all": True,
        "per_page": 1000,
        "exclude_project_id": fabric_projects
    }

    if component_types:
        query_params["component_type"] = component_types
    if component_models:
        query_params["component_model"] = component_models
    if include_sites:
        query_params["site_name"] = include_sites
    if exclude_sites:
        query_params["exclude_site_name"] = exclude_sites

    slice_run_button.disabled = True
    with slice_status_output:
        print("⏳ Querying slices with filters:")
        print(f"   Time range: {start_time} → {end_time}")
        if component_types: print(f"   Component Types: {component_types}")
        if component_models: print(f"   Component Models: {component_models}")
        if include_sites: print(f"   Include Sites: {include_sites}")
        if exclude_sites: print(f"   Exclude Sites: {exclude_sites}")

    try:
        response = api.query_slices(**query_params)
        df_slices_by_component = pd.DataFrame(response.get("data", []))

        slice_status_output.clear_output()
        with slice_status_output:
            print(f"✅ Query complete. Total slices: {response.get('total', len(df_slices_by_component))}")

        if not df_slices_by_component.empty:
            display(df_slices_by_component)
        else:
            with slice_status_output:
                print("No slices returned.")

    except Exception as e:
        with slice_status_output:
            print(f"❌ Error during query: {e}")

    slice_run_button.disabled = False

# === Bind button click ===
slice_run_button.on_click(run_slice_query)


## Active Slices containing specific components
`df_slices_by_component` dataframe generated by the above query is used.
`components_selected` specifies the chose component types.

In [ ]:
df = df_slices_by_component

# Step 2: Convert lease_start and lease_end to datetime (with UTC awareness)
# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], utc=True, format='mixed')
df['lease_end'] = pd.to_datetime(df['lease_end'], utc=True, format='mixed')

# Step 3: Define the month range for plotting
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Step 4: Count SmartNIC slices active in each month
smartnic_counts = []

for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)

    active_slices = df[
        (df['lease_start'] <= month_end) &
        (df['lease_end'] >= month_start)
    ]
    smartnic_counts.append(len(active_slices))

# Step 5: Plot the results
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), smartnic_counts)
plt.xticks(rotation=45)
plt.title(f"{components_selected} Slice Usage Per Month")
plt.xlabel("Month")
plt.ylabel("Number of Active {components_selected} Slices")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Query current Active Slices

Generates `df_slices_state` dataframe containing slices filtered by Slice State which can be used for any plotting. 

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from pytz import UTC

# === Global DataFrame ===
df_slices_state = pd.DataFrame()

# === Possible slice states ===
SLICE_STATES = [
    "Nascent", "Configuring", "StableError", "StableOK",
    "Closing", "Dead", "Modifying", "ModifyOK", "ModifyError",
    "AllocatedError", "AllocatedOK"
]

# === Default state filters ===
default_exclude_states = ["Closing", "Dead"]

# === Widgets ===
state_include_picker = widgets.SelectMultiple(
    options=SLICE_STATES,
    description="Include States:",
    rows=len(SLICE_STATES),
)

state_exclude_picker = widgets.SelectMultiple(
    options=SLICE_STATES,
    value=tuple(default_exclude_states),
    description="Exclude States:",
    rows=len(SLICE_STATES),
)

state_run_button = widgets.Button(description="Query Slices by State")
state_status_output = widgets.Output()

display(state_include_picker, state_exclude_picker, state_run_button, state_status_output)

# === Callback ===
def run_slice_state_query(b):
    global df_slices_state
    state_status_output.clear_output()

    include_states = list(state_include_picker.value)
    exclude_states = list(state_exclude_picker.value)

    query_params = {
        "fetch_all": True,
        "per_page": 1000,
        "exclude_project_id": fabric_projects
    }

    if include_states:
        query_params["slice_state"] = include_states
    if exclude_states:
        query_params["exclude_slice_state"] = exclude_states

    state_run_button.disabled = True
    with state_status_output:
        print("⏳ Querying slices with state filters:")
        if include_states:
            print(f"   Include States: {include_states}")
        if exclude_states:
            print(f"   Exclude States: {exclude_states}")

    try:
        response = api.query_slices(**query_params)
        slices = response.get("data", [])
        df_slices_state = pd.DataFrame(slices)

        state_status_output.clear_output()
        with state_status_output:
            print(f"✅ Query complete. Total slices: {len(df_slices_state)}")

        if not df_slices_state.empty:
            display(df_slices_state)
        else:
            with state_status_output:
                print("No slices returned.")

    except Exception as e:
        with state_status_output:
            print(f"❌ Error during query: {e}")

    state_run_button.disabled = False

# === Bind the button ===
state_run_button.on_click(run_slice_state_query)


## Query Slivers
`df_slivers_filtered` dataframe contains the list of slivers within the chosen time range flitered by component type, model, sliver types and sites.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, timedelta
from pytz import UTC

# === Global DataFrame ===
df_slivers_filtered = pd.DataFrame()
slivers_component_types = []
sliver_types = []
slivers = []

# === Time defaults ===
default_start = datetime(2023, 6, 1, tzinfo=UTC)
default_end = datetime.now(UTC).replace(microsecond=0)

# === Widgets ===
start_picker = widgets.DatetimePicker(description='Start:', value=default_start)
end_picker = widgets.DatetimePicker(description='End:', value=default_end)

component_type_select = widgets.SelectMultiple(
    options=["SmartNIC", "GPU", "FPGA", "Storage", "SharedNIC"],
    description="Component Type:"
)

component_model_text = widgets.Text(
    description="Component Model(s):",
    placeholder="comma-separated"
)

sliver_type_dropdown = widgets.Dropdown(
    options=[
        "VM", "Switch", "Facility", "L2STS", "L2PTP", "L2Bridge", "FABNetv4",
        "FABNetv6", "PortMirror", "L3VPN", "FABNetv4Ext", "FABNetv6Ext", "All"
    ],
    value="VM",
    description="Sliver Type:"
)

site_include_text = widgets.Text(description="Include Sites:", placeholder="comma-separated")
site_exclude_text = widgets.Text(description="Exclude Sites:", placeholder="comma-separated")

sliver_run_button = widgets.Button(description="Query Slivers")
sliver_status_output = widgets.Output()

# === Display UI ===
display(
    start_picker, end_picker,
    component_type_select, component_model_text,
    sliver_type_dropdown,
    site_include_text, site_exclude_text,
    sliver_run_button, sliver_status_output
)

# === Callback ===
def run_sliver_query(b):
    global df_slivers_filtered
    global slivers
    global slivers_component_types
    global sliver_types
    sliver_status_output.clear_output()

    start_time = start_picker.value
    end_time = end_picker.value

    if not start_time or not end_time or start_time >= end_time:
        with sliver_status_output:
            print("⚠️ Please ensure start time is before end time.")
        return

    component_types = list(component_type_select.value)
    slivers_component_types = component_types
    component_models = [m.strip() for m in component_model_text.value.split(",") if m.strip()]
    sliver_type = sliver_type_dropdown.value
    sliver_types = [sliver_type]
    include_sites = [s.strip() for s in site_include_text.value.split(",") if s.strip()]
    exclude_sites = [s.strip() for s in site_exclude_text.value.split(",") if s.strip()]

    query_params = {
        "start_time": start_time.isoformat(timespec="milliseconds"),
        "end_time": end_time.isoformat(timespec="milliseconds"),
        "fetch_all": True,
        "per_page": 1000,
        "exclude_project_id": fabric_projects
    }

    if component_types:
        query_params["component_type"] = component_types
    if component_models:
        query_params["component_model"] = component_models
    if sliver_type != "All":
        query_params["sliver_type"] = [sliver_type]
    if include_sites:
        query_params["site_name"] = include_sites
    if exclude_sites:
        query_params["exclude_site_name"] = exclude_sites

    sliver_run_button.disabled = True
    with sliver_status_output:
        print("⏳ Querying slivers with the following filters:")
        print(f"   Time: {start_time} → {end_time}")
        if component_types: print(f"   Component Types: {component_types}")
        if component_models: print(f"   Component Models: {component_models}")
        print(f"   Sliver Type: {sliver_type}")
        if include_sites: print(f"   Include Sites: {include_sites}")
        if exclude_sites: print(f"   Exclude Sites: {exclude_sites}")

    try:
        response = api.query_slivers(**query_params)
        slivers = response.get("data", [])
        df_slivers_filtered = pd.DataFrame(slivers)

        sliver_status_output.clear_output()
        with sliver_status_output:
            print(f"✅ Query complete. Total slivers: {len(df_slivers_filtered)}")

        if not df_slivers_filtered.empty:
            display(df_slivers_filtered)
        else:
            with sliver_status_output:
                print("No slivers returned.")

    except Exception as e:
        with sliver_status_output:
            print(f"❌ Error during query: {e}")

    sliver_run_button.disabled = False

# === Bind button click ===
sliver_run_button.on_click(run_sliver_query)


## Top 10 users for Components in a time range
This visualization identifies the top 10 users based on their usage of component-based slivers (e.g., SmartNICs, GPUs, FPGAs, Storage) within the specified time window. It aggregates slivers associated with hardware components and ranks users by the number of component allocations or active leases during that period. This view helps highlight which users are most actively utilizing advanced hardware resources on the testbed.

Uses `df_slivers_filtered` generated by previous query.

In [ ]:
if len(slivers_component_types):
    # {user_id: {"usage": float, "user_email": str, "project_id": str, "project_name": str}}
    user_usage = {}
    choices = [ c.lower() for c in slivers_component_types ]
    choice_label = '_'.join(choices)
    for s in slivers:
        user_id = s.get("user_id")
        user_email = s.get("user_email")
        project_id = s.get("project_id")
        project_name = s.get("project_name")
        lease_start = s.get("lease_start")
        lease_end = s.get("lease_end")
        count = 0

        for c in s.get("components", {}).get("data", []):
            if c.get("type") in choices:
                count += 1


        if not user_id or not lease_start or not lease_end:
            continue

        try:
            start = pd.to_datetime(lease_start, utc=True)
            end = pd.to_datetime(lease_end, utc=True)
            hours = max((end - start).total_seconds() / 3600.0, 0)
        except Exception:
            continue

        usage = count * hours

        if user_id not in user_usage:
            user_usage[user_id] = {
                "usage_hours": 0,
                "user_email": user_email,
                "project_id": project_id,
                "project_name": project_name
            }

        user_usage[user_id]["usage_hours"] += usage

    # Convert to DataFrame
    usage_df = pd.DataFrame([
        {
            "user_id": uid,
            "user_email": info["user_email"],
            "project_id": info["project_id"],
            "project_name": info["project_name"],
            f"{choice_label}_usage_hours": info["usage_hours"]
        }
        for uid, info in user_usage.items()
    ])

    # Sort and display top 10
    top_users = usage_df.sort_values(f"{choice_label}_usage_hours", ascending=False).head(10)
    display(top_users)

## Active Slivers per Month

In [ ]:
df = df_slivers_filtered
sliver_type_label = "_".join(sliver_types)

# Robust datetime parsing
df["lease_start"] = pd.to_datetime(df["lease_start"], format="mixed", utc=True, errors="coerce")
df["lease_end"] = pd.to_datetime(df["lease_end"], format="mixed", utc=True, errors="coerce")
df = df.dropna(subset=["lease_start", "lease_end"])

# Clamp lease times within query range
df["lease_start"] = df["lease_start"].clip(lower=query_start)
df["lease_end"] = df["lease_end"].clip(upper=query_end)

# Monthly range
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

active_counts = []
for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)
    active = df[(df["lease_start"] <= month_end) & (df["lease_end"] >= month_start)]
    active_counts.append(len(active))

# Plot
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), active_counts)
plt.xticks(rotation=45)
plt.title(f"Active {sliver_type_label} Per Month")
plt.xlabel("Month")
plt.ylabel(f"Active {sliver_type_label} Count")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

## Active Slivers per site per Month

In [ ]:
df = df_slivers_filtered
sliver_type_label = "_".join(sliver_types)

# Parse timestamps
df["lease_start"] = pd.to_datetime(df["lease_start"], format="mixed", utc=True, errors="coerce")
df["lease_end"] = pd.to_datetime(df["lease_end"], format="mixed", utc=True, errors="coerce")
df = df.dropna(subset=["lease_start", "lease_end", "site"])

df["lease_start"] = df["lease_start"].clip(lower=query_start)
df["lease_end"] = df["lease_end"].clip(upper=query_end)

# Create month range
start_month = df["lease_start"].min().to_period("M").to_timestamp()
end_month = df["lease_end"].max().to_period("M").to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq="MS")

# Initialize result DataFrame
site_names = sorted(df["site"].unique())
month_labels = month_range.strftime('%Y-%m')
usage = pd.DataFrame(0, index=month_labels, columns=site_names)

# Count active slivers per site per month
for month_start in month_range:
    month_label = month_start.strftime('%Y-%m')
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)

    active = df[(df["lease_start"] <= month_end) & (df["lease_end"] >= month_start)]
    counts = active["site"].value_counts()
    for site, count in counts.items():
        usage.at[month_label, site] += count

# Plot stacked bar
plt.figure(figsize=(14, 7))
usage.plot(kind="bar", stacked=True, width=0.8, colormap="tab20", ax=plt.gca())
plt.title(f"Active {sliver_type_label} Slivers Per Site Per Month")
plt.xlabel("Month")
plt.ylabel("Active Sliver Count")
plt.xticks(rotation=45)
plt.legend(title="Site", bbox_to_anchor=(1.01, 1), loc="upper left")
plt.tight_layout()
plt.grid(axis='y')
plt.show()